<a href="https://colab.research.google.com/github/GheorgheBriana/Big-data-faculty/blob/main/lab3_part1_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#pornim SparkContext cu 4 core-uri
from pyspark import SparkContext
sc = SparkContext(master = "local[4]")

In [14]:
#input: un fisier text ce consta din cuvinte separate prin spatii
#output: o lista de cuvinte si numarul lor de aparitii. Lista va fi
#sortata descrescator dupa numarul de aparitii

In [17]:
def pretty_print_plan(rdd):
  for x in rdd.toDebugString().decode().split('\n'):
    print(x)

In [20]:
from google.colab import files
uploaded = files.upload()


Saving Moby-Dick.txt to Moby-Dick.txt


In [24]:
%%time
text_file = sc.textFile("/content/Moby-Dick.txt")

CPU times: user 908 µs, sys: 1.01 ms, total: 1.92 ms
Wall time: 65.6 ms


In [26]:
type(text_file)


pyspark.rdd.RDD

In [ ]:
#pasii pt numararea cuvintelor
# - se separa elementele unei linii in functie de separatori (spatii)
# - se mapeaza word la (word, 1)
# - se numara aparitiile fiecarui cuvant


In [27]:
%%time
words = text_file.flatMap(lambda line: line.split(" "))
#fiecare linie din text_file se imparte in cuvinte intr-un array

#obtinere cuvinte nevide
not_empty = words.filter(lambda word: word != "") # scoate cuvintele goale, strigurile vide

#mapare word -> (word, 1) => punem valori de 1 la fiecare cuvant si dupa adunam
# pentru a numara de cate ori apare fiecare cuvant
# Spark nu stie sa numere direct cuvintele. Acesta stie sa grupeze si sa adune valori
key_values = not_empty.map(lambda word: (word, 1)) # pentru fiecare cuvant se creeaza un tuplu ("cuvant", 1)
# EXEMPLU: daca cuvintele din RDD-ul not_empty sunt: ["ana", "are", "mere"], atunci dupa mapare => [("ana", 1), ("are", 1), ("mere", 1)]

#numarare cuvinte cu reduceByKey
counts = key_values.reduceByKey(lambda x, y: x + y)


CPU times: user 14.3 ms, sys: 2.46 ms, total: 16.7 ms
Wall time: 361 ms


In [28]:
pretty_print_plan(text_file)
#output:(2) /content/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
# |  /content/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []

# => spark a impartit fisierul in 2 parti
# => MapPartitionsRDD[7] este RDD-ul principal, cel pe care se lucreaza acum (text_file). El imparte datele in partitii
# => HadoopRDD[6] este RDD-ul de baza, care citeste fisierul folosind Hadoop API

(2) /content/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [29]:
pretty_print_plan(words)

(2) PythonRDD[12] at RDD at PythonRDD.scala:53 []
 |  /content/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [30]:
pretty_print_plan(not_empty)

(2) PythonRDD[13] at RDD at PythonRDD.scala:53 []
 |  /content/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [31]:
pretty_print_plan(counts)

(2) PythonRDD[14] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[11] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[10] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[9] at reduceByKey at <timed exec>:15 []
    |  PythonRDD[8] at reduceByKey at <timed exec>:15 []
    |  /content/Moby-Dick.txt MapPartitionsRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /content/Moby-Dick.txt HadoopRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []


In [41]:
%%time
##Run #1
Count = counts.count()  #numara cate cuvinte diferite sunt

Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x + y) #x[1] = valorile frecventiale
#extrage valorile de tipul x[1] din fiecare (word, count) si le aduna pe toate
#obtine nr total de cuvinte
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 19 ms, sys: 2.74 ms, total: 21.7 ms
Wall time: 1.02 s


In [34]:
%%time
## Run #2
Count = counts.count() # nr cate cuvinte dif sunt
Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

CPU times: user 20.6 ms, sys: 5.05 ms, total: 25.7 ms
Wall time: 1.5 s


In [40]:
%%time
## Run #3, cache
Count = counts.cache().count()
Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x + y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 24.3 ms, sys: 1.19 ms, total: 25.5 ms
Wall time: 1.38 s


In [42]:
%%time
##Run #4
Count = counts.count()
Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x + y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 14.6 ms, sys: 2.27 ms, total: 16.9 ms
Wall time: 700 ms


In [43]:
%%time
#Run #5
Count = counts.count()
Sum = counts.map(lambda x: x[1]).reduce(lambda x, y: x + y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 15.4 ms, sys: 3.74 ms, total: 19.1 ms
Wall time: 1.2 s


In [45]:
sc.stop()